In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import random
import re

from tqdm import tqdm
import time

import pydicom as dicom
import nibabel as nib
import SimpleITK as sitk
import monai

import torch
import torch.nn as nn
import torch.optim as optim

from monai.networks.nets import EfficientNetBN
from monai.networks.nets import ResNet
#from efficientnet_pytorch import EfficientNet
import timm

import wandb


In [2]:
SEED = 344
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True # Fix the network according to random seed
    print('Finish seeding with seed {}'.format(seed))
    
seed_everything(SEED)
print('Training on device {}'.format(device))

Finish seeding with seed 344
Training on device cuda


In [3]:
dicom_tag_columns = [
    'Columns',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'PatientID',
    'PatientPosition',
    'PixelSpacing',
    'RescaleIntercept',
    'RescaleSlope',
    'Rows',
    'SeriesNumber',
    'SliceThickness',
    'path',
    'WindowCenter',
    'WindowWidth'
]

train_dicom_tags = pd.read_parquet('autodl-tmp/train_dicom_tags.parquet', columns=dicom_tag_columns)
test_dicom_tags = pd.read_parquet('autodl-tmp/test_dicom_tags.parquet', columns=dicom_tag_columns)

train_series_meta = pd.read_csv('autodl-tmp/train_series_meta.csv')
test_series_meta = pd.read_csv('autodl-tmp/test_series_meta.csv')

train_csv = pd.read_csv('autodl-tmp/train.csv')

mid_z_csv = pd.read_csv('middle_z.csv')
complete_series_meta = mid_z_csv[mid_z_csv.middle_z != -1].reset_index()
complete_series_meta

,index,patient_id,series_id,middle_z
0,0,10004,21057,128
1,1,10004,51033,131
2,2,10005,18667,120
3,3,10007,47578,127
4,4,10026,29700,110
...,...,...,...,...
4392,4393,9961,2003,133
4393,4394,9961,63032,129
4394,4395,9980,40214,94
4395,4396,9980,40466,151


In [4]:
injury_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 1, "patient_id"].values)]
healthy_series_meta = train_series_meta.loc[train_series_meta.patient_id.isin(train_csv.loc[train_csv.any_injury == 0, "patient_id"].values)]

In [5]:
def raw_path_gen(patient_id, series_id, train=True):
    if(train):
        path = 'autodl-tmp/train_images_resample/'
    else:
        path = 'autodl-tmp/train_images_resample/'
    
    path += str(patient_id) + '/' + str(series_id)
    
    return path

def create_3D_scans(folder, downsample_rate=1): 
    filenames = os.listdir(folder)
    filenames = [int(filename.split('.')[0]) for filename in filenames]
    filenames = sorted(filenames)
    filenames = [str(filename) + '.dcm' for filename in filenames]
        
    volume = []
    #for filename in tqdm(filenames[::downsample_rate], position=0): 
    for filename in filenames[::downsample_rate]: 
        filepath = os.path.join(folder, filename)
        ds = dicom.dcmread(filepath)
        image = ds.pixel_array
        
        if ds.PixelRepresentation == 1:
            bit_shift = ds.BitsAllocated - ds.BitsStored
            dtype = image.dtype 
            image = (image << bit_shift).astype(dtype) >>  bit_shift
        
        # find rescale params
        if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
            intercept = float(ds.RescaleIntercept)
            slope = float(ds.RescaleSlope)
    
        # find clipping params
        center = int(ds.WindowCenter)
        width = int(ds.WindowWidth)
        low = center - width / 2
        high = center + width / 2    
        
        
        image = (image * slope) + intercept
        image = np.clip(image, low, high)

        image = (image / np.max(image) * 255).astype(np.int16)
        image = image[::downsample_rate, ::downsample_rate]
        volume.append( image )
    
    volume = np.stack(volume, axis=0)
    return volume

def plot_image_with_seg(volume, volume_seg=[], orientation='Coronal', num_subplots=20):
    # simply copy
    if len(volume_seg) == 0:
        plot_mask = 0
    else:
        plot_mask = 1
        
    if orientation == 'Coronal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([1, 0, 2])
        if plot_mask:
            volume_seg = volume_seg.transpose([1, 0, 2])
        
    elif orientation == 'Sagittal':
        slices = np.linspace(0, volume.shape[2]-1, num_subplots).astype(np.int16)
        volume = volume.transpose([2, 0, 1])
        if plot_mask:
            volume_seg = volume_seg.transpose([2, 0, 1])

    elif orientation == 'Axial':
        slices = np.linspace(0, volume.shape[0]-1, num_subplots).astype(np.int16)
           
    rows = np.max( [np.floor(np.sqrt(num_subplots)).astype(int) - 2, 1])
    cols = np.ceil(num_subplots/rows).astype(int)
    
    fig, ax = plt.subplots(rows, cols, figsize=(cols * 2, rows * 4))
    fig.tight_layout(h_pad=0.01, w_pad=0)
    
    ax = ax.ravel()
    for this_ax in ax:
        this_ax.axis('off')

    for counter, this_slice in enumerate( slices ):
        plt.sca(ax[counter])
        
        image = volume[this_slice, :, :]
        plt.imshow(image, cmap='gray')
        
        if plot_mask:
            mask = np.where(volume_seg[this_slice, :, :], volume_seg[this_slice, :, :], np.nan)
            plt.imshow(mask, cmap='Set1', alpha=0.5)
            
def load_nii(patient_id, series_id, root='autodl-tmp/train_images_resample/'):
    path = root + str(patient_id) + '/' + str(series_id) + '.nii.gz'
    img = sitk.ReadImage(path)
    img = sitk.GetArrayFromImage(img)
    
    # img = nib.load(path)
    # img = img.get_fdata().transpose(2, 1, 0)
    
    return img

In [6]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

ds = 2

midz_pd = pd.read_csv('middle_z.csv')

class CTDataset(Dataset):
    def __init__(self, root='autodl-tmp/train_images_resample/', augmentation=False, meta=train_series_meta, device='cpu'):
        self.device = device
        self.series_meta = meta
        self.root = root
        self.t = monai.transforms.Compose([monai.transforms.RandZoom(prob=0.5, min_zoom=0.9, max_zoom=1.1),
                                           monai.transforms.RandRotate(range_x=3.14 / 24, prob=0.5),
                                           monai.transforms.SpatialPad(spatial_size=(320//ds, 280, 280), mode="edge"),
                                           monai.transforms.RandSpatialCrop(roi_size=(320//ds, 256, 256), random_size=False),
                                           monai.transforms.NormalizeIntensity(divisor = 400)
                                ])
        self.t_val = monai.transforms.Compose([monai.transforms.NormalizeIntensity(divisor = 400)])
        
        self.aug = augmentation
        
    def __len__(self):
        #return 1100
        return len(self.series_meta)
    
    def __getitem__(self, idx):

        patient_id, series_id = self.series_meta.loc[idx, ["patient_id", "series_id"]].astype('int')
        middle_z = midz_pd.loc[midz_pd.series_id == series_id, "middle_z"].values[0] // ds
        
        img_a = load_nii(patient_id, series_id, self.root).astype('float32')
        #img_t = torch.from_numpy(img_a).unsqueeze(0)
        #img_t = torch.from_numpy(img_a[::2, ::2, ::2]).unsqueeze(0)
        if(self.aug):
            img_t = self.t(np.expand_dims(img_a[::ds, :, :], 0))
        else:
            #img_t = torch.from_numpy(img_a[::ds, ::ds, ::ds]).unsqueeze(0)
            img_t = self.t_val(np.expand_dims(img_a[::ds, :, :], 0))
        label_columns = [
            'kidney_low',
            'kidney_high',
            'liver_low',
            'liver_high',
            'spleen_low',
            'spleen_high',
            #'any_injury'
        ]
        label_a = train_csv.loc[train_csv.patient_id == patient_id, label_columns].values[0].astype('float32')
        label_t = torch.from_numpy(label_a)
        label_midz = torch.tensor(middle_z.astype('int'))
        
        return img_t, label_t, label_midz

In [7]:
train_meta = complete_series_meta[0:3200].reset_index()
val_meta = complete_series_meta[3200:].reset_index()

# train_meta = injury_series_meta[-800:].reset_index()
# val_meta = injury_series_meta[0:-800].reset_index()

train_ds = CTDataset(meta = train_meta, augmentation=True)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=8)

val_ds = CTDataset(meta = val_meta, augmentation=False)
val_dl = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=8)

In [8]:
# class EffNet(nn.Module):
#     def __init__(self, ch_out=9):
#         super(EffNet, self).__init__()
#         #self.conv_in = nn.Conv3d(1, 3, kernel_size=5, padding=2, stride=2)
#         self.net = EfficientNetBN("efficientnet-b0", pretrained=False, progress=False, spatial_dims=3, in_channels=1, num_classes=ch_out,)
#     def forward(self, x):
#         #x = self.conv_in(x)
#         #return torch.sigmoid(self.net(x))
#         return self.net(x)

class KLSVolNet(nn.Module):
    def __init__(self, backbone = "tf_efficientnetv2_s.in21k_ft_in1k", 
                 ch_in = 3, ch_out = 9, slices = 15, dropout = 0.0, pretrained=True):
        super(KLSVolNet, self).__init__()
        self.slices = slices
        
        self.encoder = timm.create_model(
            backbone,
            in_chans=ch_in,
            num_classes=ch_out,
            features_only=False,
            drop_rate=0.0,
            drop_path_rate=0.0,
            pretrained=False,
        )
        
        if pretrained and 'efficient' in backbone:
            self.encoder.load_state_dict(torch.load('pretrained/tf_efficientnetv2_s.in21k_ft_in1k_6class.pt'))
        
        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()
        
        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=0.0, bidirectional=True, batch_first=True)
        
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(dropout),
            nn.LeakyReLU(0.1),
            nn.Linear(256, ch_out),
        )
        
        self.head2 = nn.Conv1d(slices, 1, 1)
        
        
    def slicer(self, img, mid_z, slices):
        img_kls = []
        for i in range(0, len(mid_z)):
            z = mid_z[i]
            if z < 30:
                z = 30
            elif img.shape[-3] - z < 30:
                z = img.shape[-3] - 31
                
            slice_kls = img[i, :, z-30:z+30, :, :]
            img_kls.append(slice_kls)
            # plot_image_with_seg(slice_kls[0].numpy(), orientation='Axial', num_subplots=7)
        img = torch.cat(img_kls, 0)
        img = img.unsqueeze(1)
        #print(img.shape)
        z_length = img.shape[-3]
        z_slices = (np.linspace(0, z_length, slices + 2)).astype('int')
        z_slices = z_slices[1:-1]
        #print(z_slices)
        slices_list = []
        for z in z_slices:
            slices_list.append(img[:, :, z-1:z+2, :, :])
        img_slice = torch.cat(slices_list, 1)
        return img_slice
        
    def forward(self, x, mid_z):  # (bs, nslice, ch, sz, sz)
        x = self.slicer(x, mid_z, self.slices)
        bs, nslice,ch, sz1, sz2 = x.shape
        x = x.view(bs*nslice, ch, sz1, sz2)
        
        feature_2d = self.encoder(x)
        feature_2d = feature_2d.view(bs, nslice, -1)
        
        feature_lstm, _ = self.lstm(feature_2d)
        feature_lstm = feature_lstm.contiguous().view(bs * nslice, -1)
        
        preds = self.head(feature_lstm)
        preds = preds.view(bs, nslice, -1).contiguous()
        preds = self.head2(preds)
        
        return preds.squeeze(1)
        
        
        # bs = x.shape[0]
        # x = x.view(bs * n_slice_per_c, in_chans, image_size, image_size)
        # feat = self.encoder(x)
        # feat = feat.view(bs, n_slice_per_c, -1)
        # feat, _ = self.lstm(feat)
        # feat = feat.contiguous().view(bs * n_slice_per_c, -1)
        # feat = self.head(feat)
        # feat = feat.view(bs, n_slice_per_c).contiguous()
        

In [9]:
# imgs, labels, midz = next(iter(train_dl))
# net = KLSVolNet(ch_out = 6)
# img_s = net(imgs, midz)
# img_s.shape

In [10]:
# img_a = imgs[0, 0].numpy()
# plot_image_with_seg(img_a, orientation='Coronal', num_subplots=7)

In [11]:
import sklearn.metrics

def transform_9class(label_in):
    label_out = [1 - label_in[0],
                   label_in[0],
                    1- label_in[1],
                   label_in[1],
                   (1 - label_in[2]) * (1 - label_in[3]),
                   label_in[2],
                   label_in[3],
                   (1 - label_in[4]) * (1 - label_in[5]),
                    label_in[4],
                   label_in[5],
                   (1 - label_in[6]) * (1 - label_in[7]),
                   label_in[6],
                   label_in[7]]
    return label_out

def transform_13class(label_in):
    label_out = label_in
    return label_out.tolist()

def transform_kls9class(label_in):
    label_out = [1, 1, 1, 1]
    label_out.extend(label_in.tolist())
    return label_out

def transform_kls6class(label_in):
    label_kls = label_in
    label_in = [0.5, 0.5]
    label_in.extend(label_kls.tolist())
    label_out = [1, 1, 1, 1,
                   (1 - label_in[2]) * (1 - label_in[3]),
                   label_in[2],
                   label_in[3],
                   (1 - label_in[4]) * (1 - label_in[5]),
                    label_in[4],
                   label_in[5],
                   (1 - label_in[6]) * (1 - label_in[7]),
                   label_in[6],
                   label_in[7]]
    return label_out


def loss_metrics(metrics, transform):
    preds = [transform(x) for x in metrics["predict"]]
    targets = [transform(x) for x in metrics["label"]]
    targets_any_injury = metrics["label"][:, -1]
    
    loss_list = []
    
    print("F1 score: ", sklearn.metrics.f1_score(metrics["label"], np.around(metrics["predict"]), average=None, zero_division=0.0))
    print("AUC score: ", sklearn.metrics.roc_auc_score(metrics["label"], metrics["predict"], average=None))
    
    for i in range(0, len(preds)):
        predict = preds[i]
        target = targets[i]
        
        label_pred = np.zeros(14)
        label_pred[0] = predict[0] / (predict[0] + predict[1])
        label_pred[1] = predict[1] / (predict[0] + predict[1])
        label_pred[2] = predict[2] / (predict[2] + predict[3])
        label_pred[3] = predict[3] / (predict[2] + predict[3])
        label_pred[4] = predict[4] / (predict[4] + predict[5] + predict[6])
        label_pred[5] = predict[5] / (predict[4] + predict[5] + predict[6])
        label_pred[6] = predict[6] / (predict[4] + predict[5] + predict[6])
        label_pred[7] = predict[7] / (predict[7] + predict[8] + predict[9])
        label_pred[8] = predict[8] / (predict[7] + predict[8] + predict[9])
        label_pred[9] = predict[9] / (predict[7] + predict[8] + predict[9])
        label_pred[10] = predict[10] / (predict[10] + predict[11] + predict[12])
        label_pred[11] = predict[11] / (predict[10] + predict[11] + predict[12])
        label_pred[12] = predict[12] / (predict[10] + predict[11] + predict[12])
        label_pred[13] = max([1 - label_pred[x] for x in [0, 2, 4, 7, 10]])
        
        targets_any_injury = max([1 - target[x] for x in [0, 2, 4, 7, 10]])
        
        target.append(targets_any_injury)
        label_target = np.array(target)
        
        weight = np.array([1, 2, 1, 6, 1, 2, 4, 1, 2, 4, 1, 2, 4, 6])
        
        loss_list.append(sklearn.metrics.log_loss(
            y_true=label_target,
            y_pred=label_pred,
            sample_weight=weight))
    #print("Weighted Loss: " + np.mean(loss_list))
    
    return np.mean(loss_list)
        
    
    
    #print(np.array(preds).shape)
    

In [12]:
import copy

def TrainClassifer(model,trn_dl,val_dl,optimizer, scheduler=None,
                   n_eopchs=20, device='cpu'):
 
    #loss_fn = nn.BCELoss(weight=torch.Tensor([1, 6, 1, 6, 1, 4, 8, 1, 4, 8, 1, 4, 8]).to(device))
    #loss_fn = nn.BCELoss()
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([10, 10, 6, 6, 6, 6]).to(device))
    model.to(device)
    best_model = copy.deepcopy(model)
    bestweight_model = copy.deepcopy(model)
    best_val = 999.0
    best_weightloss = 999.0
    metrics = {'predict': [], 'label' : []}
    PATH_MODEL = 'KLSVol2.5D/KLSVol2.5Dx2_Effnetv2_6class_posweight_bestloss.pt'
    PATH_MODEL2 = 'KLSVol2.5D/KLSVol2.5Dx2_Effnetv2_6class_posweight_bestmetric.pt'
    wandb.init(name='KLSVol2.5Dx2_Effnetv2_6class_posweight', 
               project='KLSVol2.5Dx2-test')

    for epoch in range(1, n_eopchs + 1):
        loss_train = 0.0
        model.train()
        for imgs, labels, midz in tqdm(trn_dl, position=0):
            imgs = imgs.to(device)
            labels = labels.to(device)
            midz = midz.to(device)

            outputs = model(imgs, midz)
            #outputs = model(imgs.unsqueeze(1))
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        torch.cuda.empty_cache()
        
        loss_val = 0.0
        correct_val = 0.0
        model.eval()
        
        for imgs, labels, midz in tqdm(val_dl):
            imgs = imgs.to(device)
            labels = labels.to(device)
            midz = midz.to(device)
            with torch.no_grad():
                outputs = model(imgs, midz)
                #outputs = model(imgs.unsqueeze(1))
                loss = loss_fn(outputs, labels)
                loss_val += loss.item()
                outputs = torch.sigmoid(outputs)
                metrics['predict'].extend((outputs.to('cpu').detach().numpy()).tolist())
                metrics['label'].extend((labels.to('cpu').detach().numpy()).tolist())
        
        metrics['predict'] = np.array(metrics['predict'])
        #metrics['predict'] = np.array([[0.5, 0.5, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333, 0.33333]]*len(metrics['label']))
        metrics['label'] = np.array(metrics['label'])
        weighted_loss = loss_metrics(metrics, transform_kls6class)
        metrics = {'predict': [], 'label' : []}
        
        torch.cuda.empty_cache()
        
        if (weighted_loss) < best_weightloss:
            best_weightloss = weighted_loss
            torch.save(model.state_dict(), 'model_tmp.pt')
            bestweight_model.load_state_dict(torch.load('model_tmp.pt'))
            
        if loss_val / len(val_dl) < best_val:
            best_val = loss_val / len(val_dl)
            torch.save(model.state_dict(), 'model_tmp.pt')
            best_model.load_state_dict(torch.load('model_tmp.pt'))
            
            
        if scheduler != None:
            scheduler.step()

        print('{} Eopch {}, Training Loss {}, Val Loss {}, Weighted Loss {}'.format(time.strftime("%Y-%m-%d %H:%M:%S",time.localtime()),
                                                                  epoch, loss_train / len(trn_dl), loss_val / len(val_dl),
                                                                                     weighted_loss))
        
        
        
        wandb.log({'training loss': loss_train / len(trn_dl),
                  'val loss': loss_val / len(val_dl),
                  'weighted loss': weighted_loss})
    torch.save(best_model.state_dict(), PATH_MODEL)
    torch.save(bestweight_model.state_dict(), PATH_MODEL2)
    print('Finish training: best_val:{}, best_weighted loss:{}'.format(best_val, best_weightloss))
    wandb.finish()

In [ ]:
net = KLSVolNet(ch_out = 6, slices = 10).to(device)

optimizer = optim.AdamW(net.parameters(), lr=1e-5)
TrainClassifer(model=net,trn_dl=train_dl,val_dl=val_dl,optimizer=optimizer, 
               scheduler=None, n_eopchs=15, device=device)

wandb: Currently logged in as: northm (rsna2023). Use `wandb login --relogin` to force relogin


100%|██████████| 300/300 [03:14<00:00,  1.54it/s]


F1 score:  [0.13186813 0.05660377 0.20357143 0.12903226 0.1414791  0.22916667]
AUC score:  [0.72160379 0.57751278 0.7177673  0.82875956 0.66054163 0.76526929]
2023-09-25 11:44:23 Eopch 1, Training Loss 0.7413182579353452, Val Loss 0.6835078802704811, Weighted Loss 0.6046326707598563


100%|██████████| 300/300 [03:18<00:00,  1.51it/s]


F1 score:  [0.15772871 0.07575758 0.23721881 0.20408163 0.14448669 0.32      ]
AUC score:  [0.79022882 0.6400637  0.74627776 0.87897196 0.68683975 0.82866278]
2023-09-25 12:00:15 Eopch 2, Training Loss 0.6198500906303525, Val Loss 0.6352266496916612, Weighted Loss 0.5656840127424922


 59%|█████▉    | 472/800 [07:33<03:08,  1.74it/s]